<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 10px; height: 55px">


# Capstone Project: Forecasting HDB Resale Prices


## Notebook 3/4: Preprocessing
---

## Getting Started

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importing relevant csv files
combined = pd.read_csv('../data/combined.csv')
nearby_amenities = pd.read_csv('../data/nearby_amenities.csv')

In [3]:
combined.head()

,date,year,month,town,flat_type,block,street_name,address,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2000-01-01,2000,1,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,170 ANG MO KIO AVE 4,07 TO 09,69.0,Standard,1986,14.0,147000.0
1,2000-01-01,2000,1,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,174 ANG MO KIO AVE 4,04 TO 06,61.0,Standard,1986,14.0,144000.0
2,2000-01-01,2000,1,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,216 ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,24.0,159000.0
3,2000-01-01,2000,1,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,215 ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,24.0,167000.0
4,2000-01-01,2000,1,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,218 ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,24.0,163000.0


In [4]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581229 entries, 0 to 581228
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   date                 581229 non-null  object 
 1   year                 581229 non-null  int64  
 2   month                581229 non-null  int64  
 3   town                 581229 non-null  object 
 4   flat_type            581229 non-null  object 
 5   block                581229 non-null  object 
 6   street_name          581229 non-null  object 
 7   address              581229 non-null  object 
 8   storey_range         581229 non-null  object 
 9   floor_area_sqm       581229 non-null  float64
 10  flat_model           581229 non-null  object 
 11  lease_commence_date  581229 non-null  int64  
 12  remaining_lease      581229 non-null  float64
 13  resale_price         581229 non-null  float64
dtypes: float64(3), int64(3), object(8)
memory usage: 62.1+ MB


In [5]:
# Converting the date column to datetime format
combined['date'] = pd.to_datetime(combined['date'])

---

## Encoding Categorical Features

The categorical features of our dataset need to be encoded as machine learning models require all input and output variables to be numeric. We will encode each feature differently depending on its characteristics.

### Categorical Feature 1: Town

It is quite difficult to determine if towns are ranked in a natural order. One of the most common ways towns are evaluated are by their distance to the central region of Singapore, and this is already taken into account in our engineered feature, `cityhall_dist`. Hence, we will use one-hot encoding for the `town` feature.

In [6]:
# Getting dummy variables for the 'town' column
combined = pd.get_dummies(data = combined, columns = ['town'], drop_first = True, prefix = 'town')

In [7]:
# Converting column names to lowercase
combined.columns = [i.lower() for i in combined.columns]

In [8]:
combined.head()

,date,year,month,flat_type,block,street_name,address,storey_range,floor_area_sqm,flat_model,...,town_pasir ris,town_punggol,town_queenstown,town_sembawang,town_sengkang,town_serangoon,town_tampines,town_toa payoh,town_woodlands,town_yishun
0,2000-01-01,2000,1,3 ROOM,170,ANG MO KIO AVE 4,170 ANG MO KIO AVE 4,07 TO 09,69.0,Standard,...,0,0,0,0,0,0,0,0,0,0
1,2000-01-01,2000,1,3 ROOM,174,ANG MO KIO AVE 4,174 ANG MO KIO AVE 4,04 TO 06,61.0,Standard,...,0,0,0,0,0,0,0,0,0,0
2,2000-01-01,2000,1,3 ROOM,216,ANG MO KIO AVE 1,216 ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,...,0,0,0,0,0,0,0,0,0,0
3,2000-01-01,2000,1,3 ROOM,215,ANG MO KIO AVE 1,215 ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,...,0,0,0,0,0,0,0,0,0,0
4,2000-01-01,2000,1,3 ROOM,218,ANG MO KIO AVE 1,218 ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,...,0,0,0,0,0,0,0,0,0,0


### Categorical Feature 2: Flat Model

In the earlier stages of our project, we have already narrowed down the different categories of flat models. From these 6 categories, we can easily see that 'Apartment', 'Maisonette' and 'Special' consist of flats that are considered more premium. We can expect these 'premium' flats to sell for higher prices. Hence, ordinal encoding should be used.

In [9]:
# Getting the different categories of `flat_model`
combined['flat_model'].unique()

array(['Standard', 'New Generation', 'Model A', 'Apartment', 'Maisonette',
       'Special'], dtype=object)

In [10]:
# Ordinal encoding of the `flat_model` column
combined['flat_model'] = combined['flat_model'].map({'Standard': 1, 
                                                     'New Generation': 1,
                                                     'Model A': 1,
                                                     'Apartment': 2, 
                                                     'Maisonette': 2, 
                                                     'Special': 2})

In [11]:
print(combined.shape)
combined.head()

(581229, 38)


,date,year,month,flat_type,block,street_name,address,storey_range,floor_area_sqm,flat_model,...,town_pasir ris,town_punggol,town_queenstown,town_sembawang,town_sengkang,town_serangoon,town_tampines,town_toa payoh,town_woodlands,town_yishun
0,2000-01-01,2000,1,3 ROOM,170,ANG MO KIO AVE 4,170 ANG MO KIO AVE 4,07 TO 09,69.0,1,...,0,0,0,0,0,0,0,0,0,0
1,2000-01-01,2000,1,3 ROOM,174,ANG MO KIO AVE 4,174 ANG MO KIO AVE 4,04 TO 06,61.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2000-01-01,2000,1,3 ROOM,216,ANG MO KIO AVE 1,216 ANG MO KIO AVE 1,07 TO 09,73.0,1,...,0,0,0,0,0,0,0,0,0,0
3,2000-01-01,2000,1,3 ROOM,215,ANG MO KIO AVE 1,215 ANG MO KIO AVE 1,07 TO 09,73.0,1,...,0,0,0,0,0,0,0,0,0,0
4,2000-01-01,2000,1,3 ROOM,218,ANG MO KIO AVE 1,218 ANG MO KIO AVE 1,07 TO 09,67.0,1,...,0,0,0,0,0,0,0,0,0,0


### Categorical Feature 3: Storey Range

Flats on higher floors generally cost more than flats on lower floors. Hence we should encode `storey_range` in a way that reflects its ranking.

In [12]:
# Displaying the ranks for each category of 'storey_range'
dict(enumerate(combined.storey_range.astype('category').cat.categories))

{0: '01 TO 03',
 1: '01 TO 05',
 2: '04 TO 06',
 3: '06 TO 10',
 4: '07 TO 09',
 5: '10 TO 12',
 6: '11 TO 15',
 7: '13 TO 15',
 8: '16 TO 18',
 9: '16 TO 20',
 10: '19 TO 21',
 11: '21 TO 25',
 12: '22 TO 24',
 13: '25 TO 27',
 14: '26 TO 30',
 15: '28 TO 30',
 16: '31 TO 33',
 17: '31 TO 35',
 18: '34 TO 36',
 19: '36 TO 40',
 20: '37 TO 39',
 21: '40 TO 42',
 22: '43 TO 45',
 23: '46 TO 48',
 24: '49 TO 51'}

In [13]:
# Creating a new column for 'storey_cat' which reflects the rank of 'storey_range'
combined['storey_cat'] = combined['storey_range'].astype('category').cat.codes

# Dropping the 'storey_range' column
combined = combined.drop(columns = ['storey_range'])

In [14]:
print(combined.shape)
combined.head()

(581229, 38)


,date,year,month,flat_type,block,street_name,address,floor_area_sqm,flat_model,lease_commence_date,...,town_punggol,town_queenstown,town_sembawang,town_sengkang,town_serangoon,town_tampines,town_toa payoh,town_woodlands,town_yishun,storey_cat
0,2000-01-01,2000,1,3 ROOM,170,ANG MO KIO AVE 4,170 ANG MO KIO AVE 4,69.0,1,1986,...,0,0,0,0,0,0,0,0,0,4
1,2000-01-01,2000,1,3 ROOM,174,ANG MO KIO AVE 4,174 ANG MO KIO AVE 4,61.0,1,1986,...,0,0,0,0,0,0,0,0,0,2
2,2000-01-01,2000,1,3 ROOM,216,ANG MO KIO AVE 1,216 ANG MO KIO AVE 1,73.0,1,1976,...,0,0,0,0,0,0,0,0,0,4
3,2000-01-01,2000,1,3 ROOM,215,ANG MO KIO AVE 1,215 ANG MO KIO AVE 1,73.0,1,1976,...,0,0,0,0,0,0,0,0,0,4
4,2000-01-01,2000,1,3 ROOM,218,ANG MO KIO AVE 1,218 ANG MO KIO AVE 1,67.0,1,1976,...,0,0,0,0,0,0,0,0,0,4


### Categorical Feature 4: Flat Type

Flat type also has an obvious rank as it usually determines the square footage and the amount of rooms in a flat. Hence, ordinal encoding will be utilised.

In [15]:
# Displaying the categories for 'flat_type'
combined['flat_type'].unique()

array(['3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE'], dtype=object)

In [16]:
# Assigning a numerical value to each category of 'flat_type'
combined['flat_type'] = combined['flat_type'].map({'3 ROOM': 1, '4 ROOM': 2,'5 ROOM': 3,'EXECUTIVE': 4})

In [17]:
print(combined.shape)
combined.head()


(581229, 38)


,date,year,month,flat_type,block,street_name,address,floor_area_sqm,flat_model,lease_commence_date,...,town_punggol,town_queenstown,town_sembawang,town_sengkang,town_serangoon,town_tampines,town_toa payoh,town_woodlands,town_yishun,storey_cat
0,2000-01-01,2000,1,1,170,ANG MO KIO AVE 4,170 ANG MO KIO AVE 4,69.0,1,1986,...,0,0,0,0,0,0,0,0,0,4
1,2000-01-01,2000,1,1,174,ANG MO KIO AVE 4,174 ANG MO KIO AVE 4,61.0,1,1986,...,0,0,0,0,0,0,0,0,0,2
2,2000-01-01,2000,1,1,216,ANG MO KIO AVE 1,216 ANG MO KIO AVE 1,73.0,1,1976,...,0,0,0,0,0,0,0,0,0,4
3,2000-01-01,2000,1,1,215,ANG MO KIO AVE 1,215 ANG MO KIO AVE 1,73.0,1,1976,...,0,0,0,0,0,0,0,0,0,4
4,2000-01-01,2000,1,1,218,ANG MO KIO AVE 1,218 ANG MO KIO AVE 1,67.0,1,1976,...,0,0,0,0,0,0,0,0,0,4


---

## Merging with Nearby Amenities Dataset

Finally, we need to include our previously engineered features into our main dataset. (The `nearby_amenities` dataset contains information for individual blocks of flats.)

In [18]:
print(nearby_amenities.shape)
nearby_amenities.head()

(9462, 19)


,address,mrt,mrt_dist,num_mrt_1km,mall,mall_dist,num_mall_1km,supermarket,supermarket_dist,num_supermarket_1km,hawker_dist,num_hawker_1km,park_dist,num_park_1km,school,school_dist,num_school_1km,number_school_btw_1km_2km,cityhall_dist
0,148 WOODLANDS ST 13,MARSILING MRT STATION,0.359246,1,Marsiling Mall,0.695939,1,730182,0.231658,5,0.685467,2,0.500810,2,738927,0.217030,2,5,18.000426
1,426 CLEMENTI AVE 3,CLEMENTI MRT STATION,0.392851,1,321 Clementi,0.098085,4,120451,0.172466,8,0.188672,5,0.318873,5,129903,0.402997,4,0,9.982797
2,85 CIRCUIT RD,MATTAR MRT STATION,0.544896,6,Paya Lebar Square,0.870095,1,380114,0.156163,8,0.094512,5,0.233353,2,387724,0.498281,3,3,4.973693
3,467 TAMPINES ST 44,TAMPINES EAST MRT STATION,0.417710,1,Loyang Point,1.317606,0,520475,0.175411,5,1.528611,0,0.795132,1,529565,0.658775,3,8,13.604509
4,367 CORPORATION DR,LAKESIDE MRT STATION,0.753707,1,Taman Jurong Shopping Centre,0.347819,1,610399,0.319123,2,0.433490,1,0.269495,2,618310,0.130183,2,5,15.586738


In [19]:
# Merging both datasets on 'address'
final = combined.merge(nearby_amenities, on='address', how='left')

In [20]:
print(final.shape)
final.head()

(581229, 56)


,date,year,month,flat_type,block,street_name,address,floor_area_sqm,flat_model,lease_commence_date,...,num_supermarket_1km,hawker_dist,num_hawker_1km,park_dist,num_park_1km,school,school_dist,num_school_1km,number_school_btw_1km_2km,cityhall_dist
0,2000-01-01,2000,1,1,170,ANG MO KIO AVE 4,170 ANG MO KIO AVE 4,69.0,1,1986,...,5,0.313910,5,0.306142,11,569948,0.247264,3,4,9.107076
1,2000-01-01,2000,1,1,174,ANG MO KIO AVE 4,174 ANG MO KIO AVE 4,61.0,1,1986,...,5,0.189372,4,0.340958,11,569948,0.232789,3,4,9.202219
2,2000-01-01,2000,1,1,216,ANG MO KIO AVE 1,216 ANG MO KIO AVE 1,73.0,1,1976,...,9,0.207364,5,0.752762,4,569920,0.341979,1,8,8.161709
3,2000-01-01,2000,1,1,215,ANG MO KIO AVE 1,215 ANG MO KIO AVE 1,73.0,1,1976,...,9,0.227171,5,0.734493,3,569920,0.342322,1,8,8.188099
4,2000-01-01,2000,1,1,218,ANG MO KIO AVE 1,218 ANG MO KIO AVE 1,67.0,1,1976,...,9,0.307044,4,0.739071,4,569920,0.456619,1,7,8.039989


In [21]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581229 entries, 0 to 581228
Data columns (total 56 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   date                       581229 non-null  datetime64[ns]
 1   year                       581229 non-null  int64         
 2   month                      581229 non-null  int64         
 3   flat_type                  581229 non-null  int64         
 4   block                      581229 non-null  object        
 5   street_name                581229 non-null  object        
 6   address                    581229 non-null  object        
 7   floor_area_sqm             581229 non-null  float64       
 8   flat_model                 581229 non-null  int64         
 9   lease_commence_date        581229 non-null  int64         
 10  remaining_lease            581229 non-null  float64       
 11  resale_price               581229 non-null  float64 

In [22]:
# Exporting final dataset to csv
final.to_csv('../data/final.csv', index=False)

---